## Logistic Regression Exercise

In this exercise we will perform linear regression and logistic regression on the Titanic dataset. This dataset contains information about passenger characteristics, and whether each passenger survived the disaster.

In [ ]:
# Don't change this cell; just run it.
import numpy as np
import pandas as pd
# Safe settings for Pandas.
pd.set_option('mode.chained_assignment', 'raise')
%matplotlib inline
import matplotlib.pyplot as plt
# Make the plots look more fancy.
plt.style.use('fivethirtyeight')
# Optimization function
from scipy.optimize import minimize

First let's import the data.

In [ ]:
titanic = pd.read_csv('titanic_clean.csv')
titanic.head()

## Linear Regression with one predictor

We are going to predict `survived` from `fare`. Were passengers who paid a
higher fare more likely to survive?

First, let's see how well a linear regression model does at predicting
`survived` from `fare`.

### Dummify

Create a dummy variable for `survived` and add it to the `titanic` dataframe.
We will treat `survived == 'yes'` as the event of interest, so give this the
label `1` (the integer 1). Label `survived == 'no'` with a integer value of
`0`.

In [ ]:
titanic['survived_dummy'] = ...
# Show the first few rows of the dataframe.
titanic.head()

### Column convenience

Store the data from the `fare` and `survived_dummy` columns as separate
variables.  Don't forget your [safe-handling of Pandas](https://uob-ds.github.io/cfd2021/wild-pandas/safe_pandas).

In [ ]:
fare = ...
# Show the first few values.
fare

In [ ]:
survived_dummy = ...
# Show the first few values.
survived_dummy.head()

### Plotting

To get a graphical idea of how `fare` relates to `survived`, let's plot them
together.

Create a scatterplot with `fare` on the x axis and `survived_dummy` on the y
axis:

In [ ]:
...

*Question 5:*

Do you think it looks like there is a relationship between these two variables?
How would you describe the relationship? (Rememeber that `survived_dummy == 1`
means the passenger survived, `survived_dummy == 0` means they did not). 

*Edit this cell with your answer ...*


### Minimizing sum of squares

The first step is to do a basic standard simple linear regression.

The linear regression will use a straight line (intercept and slope) to predict
the y values (`survived_dummy`) using the x values (`fare`).

We start by grabbing a version of the `ss_any_line` function from the
[using minimize page](../mean-slopes/using_minimize).

In [ ]:
def ss_any_line(c_s, x_values, y_values):
    """ Cost function for sum of squares given (intercept, slope), x, y

    Parameters
    ----------
    c_s : sequence
        List or array with two elements.  The first is the intercept to calculate for, the second is the slope.
    x_values : array or Series
        x values to predict from.
    y_values : array or Series
        y_values to predict, using intercept, slope from `c_s` and `x_values`.

    Returns
    -------
    ss_error : float
        Sum of the squared difference between predictions and `y_values`.
    """
    # c_s is a list containing two elements, an intercept and a slope.
    intercept, slope = c_s
    # Values predicted from these x_values, using this intercept and slope.
    predicted = intercept + x_values * slope
    # Difference of prediction from the actual y values.
    error = y_values - predicted
    # Sum of squared error.
    return np.sum(error ** 2)

Now get the results from applying `minimize` to this function, and specifying
the x and y values (`fare` and `survived_dummy`). These results will include,
among other things, the intercept and slope pair which minimizes the sum of the
squared error.

*Hint*: Remember the [using minimize page](../mean-slopes/using_minimize).

*Hint*: The first cell in this notebook should have imported the `minimize`
function.

In [ ]:
lin_reg_one = ...
# Show the result object from minimize.
# This should be the whole results object, including the `fun` and `x`
# attributes.
lin_reg_one

### First pass intercept and slope

We store the intercept and slope from `minimize` as separate variables:

In [ ]:
lin_reg_one_intercept, lin_reg_one_slope = lin_reg_one.x
print('Intercept from minimize (linear regression) =', lin_reg_one_intercept)
print('Slope from minimize (linear regression) =', lin_reg_one_slope)

### First pass and Statsmodels

Let's compare the intercept and slope from `minimize` to the intercept and
slope we get from Statsmodels. Complete the code in the cell below to predict
`survived` from `fare` using Statsmodels linear regression (check under `coef`
on the model summary shown below the cell, and compare these to the
intercept/slope from minimize):

*Hint*: you may want to check the [Simple and multiple regression page](https://uob-ds.github.io/cfd2021/classification/single_multiple) for a refresher on how to do
this.

*Hint*: `smf.ols()` is one way of performing linear regression with
Statsmodels; `ols` stands for Ordinary Least Squares.

In [ ]:
# The formula interface to Statsmodels
import statsmodels.formula.api as smf

# Specify the model
mod = smf.ols(...)
#- Fit the model
mod_fit = ...
# Show the model summary.
mod_fit.summary()

How do the slope and intercept from Statsmodels compare to the ones we got from
`minimize` (below).

In [ ]:
# Run this cell to view the intercept/slope we got from minimize.
print('Intercept from minimize (linear regression) =', lin_reg_one_intercept)
print('Slope from minimize (linear regression) =', lin_reg_one_slope)

### First predictions

Recall that, when we use linear regression to predict probabilities, the predicted probability is calculated from:

$
\text{predicted probability} = intercept + slope * \text{fare}
$

Use the intercept and slope we got from `minimize` to calculate the predicted
probability of survival. Store the result in a variable called
`predictions_lin_reg_one`.

In [ ]:
predictions_lin_reg_one = ...
# Show the result
predictions_lin_reg_one

Now we plot these predictions against the original data:

In [ ]:
# The original x and y variables.
titanic.plot.scatter('fare', 'survived_dummy')
# Add linear regression predictions to the graph.
plt.scatter(fare, predictions_lin_reg_one, color = 'gold');

You should see from the graph that for a `fare` of around 500, our linear
regression model predicts a probability of survival greater than 1.

Let's see if logistic regression does a better job.


## Logistic Regression with one predictor

Before we begin, here is a brief recap of the difference between probability
and odds ratios.   See the *probability and odds* section of the [logistic
regression page](https://uob-ds.github.io/cfd2021/more-regression/logistic_regression) for
background.

The output of the cell below shows some of the values from `class` column of
the `titanic` Dataframe. These values are stored in a variable called
`some_class_data`:

In [ ]:
# Some elements of the 'class' column in the titanic Dataframe
some_class_data = titanic['class'].loc[100:109]
some_class_data

### Classy probability

In the `some_class_data` series, what is the probability of a passenger being
in `1st` class? 

Remember that probability is $\frac{\text{number of observations of interest}}{\text{total number of observations}}$

Calculate the probability of being `1st` class in the `some_class_data` series.
Store the result in a variable called `prob_first`.

In [ ]:
prob_first = ...
# Show the result.
prob_first

### Classy odds

In the `some_class_data` series, what is the odds ratio of being `1st` class?

Remember that if $p$ is the probability of the event of interest, the odds
ratio of the event of interest is $\frac{p}{1 - p}$

Calculate the odds ratio of being `1st` class in the `some_class_data` series, store the result in a variable called `odds_first`:

In [ ]:
odds_first = ...
# Show the result.
odds_first

Remember that logistic regression predicts the *log odds ratio* of the event of
interest.

To perform our logistic regression, we define the functions we need.  These
functions from the [logistic regression page](https://uob-ds.github.io/cfd2021/more-regression/logistic_regression). Read over each line and make sure you
understand what it is doing.

In [ ]:
# Functions for logistic regression

def inv_logit(y):
    """ Reverse logit transformation
    """
    # Reverse the log operation.
    odds_ratios = np.exp(y)
    # Reverse odds ratios operation and return probabilities.
    return odds_ratios / (odds_ratios + 1)


def mll_logit_cost(intercept_and_slope, x, y):
    """ Cost function for maximum log likelihood

    Return minus of the log of the likelihood.
    """
    intercept, slope = intercept_and_slope

    # Make predictions for sigmoid.
    predicted_log_odds = intercept + slope * x
    pp_of_1 = inv_logit(predicted_log_odds)

    # Calculate predicted probabilities of the actual labels.
    pp_of_correct_label = y * pp_of_1 + (1 - y) * (1 - pp_of_1)

    # Use logs to calculate log of the likelihood
    log_likelihood = np.sum(np.log(pp_of_correct_label))

    # Ask minimize to find maximum by adding minus sign.
    return -log_likelihood

### Logistic minimize

Apply `minimize` to get the minimize result object.  The results include `x` —
the intercept and slope pair that gives the smallest negative log likelihood,
when we predict `survived_dummy` from `fare`. Store the results in a variable
called `log_reg_one`:

*Hint*: begin with a guessed intercept and guessed slope both around 0.1,
otherwise the function may fail...

*Hint*: remember you will need to specify your x and y values, in the call to `minimize`, as you did above.

In [ ]:
log_reg_one = ...
log_reg_one

As above, we store the intercept and slope as separate variables. Call the
intercept `log_reg_one_intercept` and call the slope `log_reg_one_slope`:

In [ ]:
log_reg_one_intercept, log_reg_one_slope = log_reg_one.x
print('Intercept from minimize (logistic regression) =', log_reg_one_intercept)
print('Slope from minimize (logistic regression) =', log_reg_one_slope)

### Logistics and Statsmodels

Let us compare these new intercept and slope values to those we get from
Statsmodels.

Perform a logistic regression, using `smf.logit()`, to predict `survived_dummy` from `fare`:

In [ ]:
log_mod = ...
log_mod_fit = ...
# Show the summary of the model fit.
log_mod_fit.summary()

How do the intercept and slope from Statsmodels compare to the ones we got from
`minimize`?

In [ ]:
# run this cell to show the intercept and slope we got from minimize
print('Intercept from minimize (logistic regression) =', log_reg_one_intercept)
print('Slope from minimize (logistic regression) =', log_reg_one_slope)

### Log-odds predictions

We can use the intercept and slope from logistic regression to predict the log
odds ratio of survival, using:

$ \text{predicted log odds ratio} = intercept + slope * \text{fare} $

Calculate the predicted log odds ratio of survival for each passenger. Store
these predictions in a variable called `predictions_log_reg_one`:

In [ ]:
predictions_log_reg_one = ...
# Show the results.
predictions_log_reg_one

### Probability predictions

The predicted log odds ratios are hard to interpret. Convert them to
probabilities, and store the probabilities in the ` predictions_log_reg_one`
variable.

*Hint* : you already have the functions you need to do this.

In [ ]:
predictions_log_reg_one = ...
# Show the results.
predictions_log_reg_one

Let's plot the probabilities, alongside the original data.

We create another scatterplot with `fare` on the x axis and `survived_dummy` on
the y axis.

In [ ]:
# Plot the logistic reg probabilities
titanic.plot.scatter('fare', 'survived_dummy')
plt.scatter(fare, predictions_log_reg_one, color='gold');

Do you think this model gives a better fit than the linear regression model?
Write your answer in the cell below:

*Edit this cell with your answer ...*